### Tutorial on Autogen

This tutorial is based on the previous ones in the github repo [here](https://github.com/sugarforever/LangChain-Advanced/tree/main/Integrations/AutoGen)

This tutorial runs successfully with **pyautogen==0.2.0**, **langchain==0.0.352**, **openai==1.6.1**, **pydantic==1.10.12**

**AutoGen + LangChain Use Case - Uniswap Protocol AI Agent**

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation. They are adaptable to different operation modes that encompass the utilization of LLMs, human inputs, and various tools.

**`LangChain`** is an open-source framework designed for software developers engaged in AI and ML. It enables them to seamlessly integrate LLM with external components, facilitating the creation of LLM-driven applications. The primary aim of LangChain is to establish connections between LLMs such as OpenAI's GPT-3.5 and GPT-4 and various external data sources, enabling the development and utilization of NLP applications.

Both of them are playing key roles in the LLM application development.

**NOTE**:

`AutoGen` already supports RAG itself, but it's more flexible to cooperate with the expert `langchain` 

`Uniswap` is a decentralized exchange that allows users to trade Ethereum-based tokens.

#### step1. prepare the enviroment

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # the local .env file should involve the OPENAI_API_KEY to use openai models

True

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

import autogen

#### step2. build up a vector store with Uniswap V3 whitepaper.

the pdf file used below can be downloaded [here](https://uniswap.org/whitepaper-v3.pdf)

In [4]:
loaders = [ PyPDFLoader('./data/uniswap_v3.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)
len(docs), docs

(51,
 [Document(page_content='Uniswap v3 Core\nMarch 2021\nHayden Adams\nhayden@uniswap.orgNoah Zinsmeister\nnoah@uniswap.orgMoody Salem\nmoody@uniswap.org\nRiver Keefer\nriver@uniswap.orgDan Robinson\ndan@paradigm.xyz\nABSTRACT\nUniswap v3 is a noncustodial automated market maker imple-\nmented for the Ethereum Virtual Machine. In comparison to earlier\nversions of the protocol, Uniswap v3 provides increased capital\nefficiency and fine-tuned control to liquidity providers, improves\nthe accuracy and convenience of the price oracle, and has a more\nflexible fee structure.\n1 INTRODUCTION\nAutomated market makers (AMMs) are agents that pool liquidity\nand make it available to traders according to an algorithm [ 5]. Con-\nstant function market makers (CFMMs), a broad class of AMMs of\nwhich Uniswap is a member, have seen widespread use in the con-\ntext of decentralized finance, where they are typically implemented\nas smart contracts that trade tokens on a permissionless blockchain\n[2

In [5]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs)
vectorstore

#### step3. set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [6]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [7]:
result = qa(({"question": "What is uniswap?"}))

In [8]:
result

{'question': 'What is uniswap?',
 'chat_history': [HumanMessage(content='What is uniswap?'),
  AIMessage(content=' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, improves the accuracy and convenience of the price oracle, and has a more flexible fee structure. It was created by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, and Dan Robinson in March 2021.')],
 'answer': ' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, improves the accuracy and convenience of the price oracle, and has a more flexible fee structure. It was created by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, and Dan Robinson in March 2021.'}

In [9]:
print(result['answer'])

 Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, improves the accuracy and convenience of the price oracle, and has a more flexible fee structure. It was created by Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, and Dan Robinson in March 2021.


#### step4. define a function `answer_uniswap_question`

It takes a parameter `question`, calls the QA chain, and answer it by returning the answer from the chain response.

In [10]:
def answer_uniswap_question(question):
  response = qa({"question": question})
  return response["answer"]

#### step5. set up AutoGen user agent and assistant agent with function calling enabled.

In [11]:
config_list = [{
    "model": "gpt-4",
    "api_key": os.getenv('OPENAI_API_KEY')
}]


llm_config={
    # "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [12]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_uniswap_question": answer_uniswap_question}
)

#### step6. use the user agent to ask the agents to write an introduction blog for `Uniswap` protocol v3.

In [13]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_uniswap_question *****
Arguments: 

{
  "question": "What is Uniswap?"
}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_uniswap_question...
user_proxy (to assistant):

***** Response from calling function "answer_uniswap_question" *****
 Uniswap v3 is a noncustodial automated market maker implemented for the Ethereum Virtual Machine. It provides increased capital efficiency and fine-tuned control to liquidity providers, i

#### step7. directly use the RAG component built in autogen

In [21]:
from chromadb.utils import embedding_functions

openai_embedding_function = embedding_functions.OpenAIEmbeddingFunction(api_key = os.getenv('OPENAI_API_KEY'))

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\r", "\t"])

In [24]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

llm_config = {
    # "request_timeout": 600,
    "config_list": config_list,
    "temperature": 0
}

In [27]:
rag_assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

rag_agent = RetrieveUserProxyAgent(
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "./data/uniswap_v3.pdf",
        "collection_name": "rag_col",
        "embedding_function": openai_embedding_function,
        "custom_text_split_function": text_splitter.split_text,
    },
)

In [28]:
rag_assistant.reset()
rag_agent.initiate_chat(
    rag_assistant, 
    problem="What are the main changes in Uniswap version 3", 
    n_results=2)

Trying to create collection.


doc_ids:  [['doc_0', 'doc_2']]
Adding doc_id doc_0 to context.
Adding doc_id doc_2 to context.
RetrieveChatAgent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What are the main changes in Uniswap version 3

Context is: Uniswap v3 Core
March 2021
Hayden Adams
hayden@uniswap.orgNoah Zinsmeister
noah@uniswap.orgMoody Salem
moody@uniswap.org
River Keefer
river@uniswap.orgDan Robinson
dan@paradigm.xyz
ABSTRACT
Uniswap v3 is a noncustodial automated market maker imple-
mented for the Ethereum Virtual Machine. In comparison to earlier
versions of the protocol, Uniswap v3 provides increased capital
efficiency and fine-tuned control to liquidity providers, improves
the accuracy and convenience of the price oracle, and